In [40]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions

spark = SparkSession.builder\
    .master("spark://192.168.2.59:7077")\
    .appName("test_app")\
    .config("spark.driver.memory","4096m")\
    .config("spark.executor.memory", "2048m")\
    .getOrCreate()

In [3]:
rc = spark.read.format('json').load('hdfs:///reddit_comments')

In [4]:
rc.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- created_utc: string (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: long (nullable = true)
 |-- edited: string (nullable = true)
 |-- gilded: long (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- removal_reason: string (nullable = true)
 |-- retrieved_on: long (nullable = true)
 |-- score: long (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: long (nullable = true)



In [8]:
rc.count()

5689002

In [5]:
df_subreddits = rc.groupBy('subreddit')
df_subreddits.count().orderBy("count", ascending=False).show()

+-------------------+------+
|          subreddit| count|
+-------------------+------+
|          AskReddit|875973|
|               pics|487514|
|         reddit.com|325653|
|             gaming|244828|
|           politics|243931|
|               IAmA|210891|
|                WTF|196335|
|              funny|165341|
|            atheism|139363|
|              trees|131064|
|fffffffuuuuuuuuuuuu|106991|
|          worldnews| 80233|
|          Minecraft| 75818|
|            science| 66700|
|        programming| 57470|
|          starcraft| 55682|
|             videos| 54423|
|                sex| 54348|
|    DoesAnybodyElse| 52189|
|      todayilearned| 47765|
+-------------------+------+
only showing top 20 rows



In [78]:
df_askreddit = rc.filter(rc["subreddit"] == "AskReddit")

df_askreddit_words = df_askreddit.withColumn('word',(functions.explode(functions.split(functions.col('body'), ' '))))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)

lowercase_askreddit_words = df_askreddit_words.withColumn("word", functions.lower(df_askreddit_words["word"]))
lowercase_askreddit_words.show()

+----+------+
|word| count|
+----+------+
| the|885481|
|  to|732147|
|   a|692565|
|   i|662375|
| and|590956|
|    |527078|
|  of|462674|
| you|382370|
|  in|327330|
|that|326638|
|  is|301202|
|  it|264385|
| for|236848|
|  my|193792|
| was|187433|
|have|182522|
|with|179988|
|  on|170839|
|  be|166245|
| but|154472|
+----+------+
only showing top 20 rows



In [54]:
df_askreddit = rc.filter(rc["subreddit"] == "pics")

df_pics_words = df_askreddit.withColumn('word', functions.explode(functions.split(functions.col('body'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)

In [62]:
profanity= ["asshole","bitch","bloody","bollocks","bugger","bullshit","bitch","cock","cocksucker","coonass","Cornhole","Cracker"
            ,"Cunt","Dick","Dickhead","Faggot","Fuck","Motherfucker","Nigga","Nigger","Paki","Pussy","Shit","Slut","Tranny"
            "Twat","Wanker"]

ask_reddit_count = 0
pics_count = 0

askreddit_profanity = df_askreddit_words.filter(df_askreddit_words["word"].isin(profanity))

askreddit_profanity.show()



+------------+-----+
|        word|count|
+------------+-----+
|        Fuck| 2631|
|        Shit|  564|
|        Dick|  209|
|       Bitch|   86|
|      Bloody|   75|
|    Bullshit|   36|
|        Cock|   33|
|       Pussy|   31|
|     Asshole|   27|
|        Cunt|   24|
|      Nigger|   14|
|     Cracker|   12|
|        Slut|   12|
|        Paki|   10|
|Motherfucker|   10|
|    Dickhead|    9|
|    Bollocks|    5|
|       Nigga|    5|
|      Faggot|    3|
|      Bugger|    1|
+------------+-----+

